In [ ]:
import os
import random
import subprocess
from eppy import modeleditor
from eppy.modeleditor import IDF
from SALib.sample import latin

In [ ]:
# Stellen Sie sicher, dass der Pfad zu EnergyPlus IDD-Datei korrekt ist
idd_path = '/usr/local/EnergyPlus-9-4-0/Energy+.idd'
IDF.setiddname(idd_path)

In [ ]:
# Funktion zum Anpassen der IDF-Datei
def modify_idf(template_path, output_path, cooling_setpoint, people_in_room, cooling_load):
    idf = IDF(template_path)
    print(template_path)
    
    # Cooling Setpoint ändern
    for obj in idf.idfobjects['HVACTemplate:Thermostat']:
        obj.Constant_Cooling_Setpoint = cooling_setpoint
    
    # People in Room ändern
    for obj in idf.idfobjects['PEOPLE']:
        obj.Number_of_People = people_in_room
    
    # Cooling Load ändern (z.B., durch Ändern von Equipment oder Zone Sizing)
    for obj in idf.idfobjects['HVACTemplate:Zone:IdealLoadsAirSystem']:
        obj.Maximum_Total_Cooling_Capacity = cooling_load
    
    # Geänderte IDF-Datei speichern
    idf.save(output_path)

# Funktion zum Ausführen der Simulation
def run_simulation(idf_path, output_dir):
    # command = f"energyplus -r -x -w AUT_ST_Graz.Univ.112900_TMYx.epw -d {output_dir} {idf_path}"
    command = "energyplus -r -x -w AUT_ST_Graz.Univ_Ensemble_ssp585_2050.epw -d " + output_dir + " " + idf_path
    # command = "energyplus", "-r", "-x", "-w", "AUT_ST_Graz.Univ.112900_TMYx.epw", "-d", output_dir , idf_path
    print(command)
    subprocess.run(command, shell=True)


In [ ]:
# Pfade
# template_idf_path = "/home/burns/PycharmProjects/MasterThesis/data/IDFs/final_office_IdealLoad_summer.idf"
template_idf_path = "/home/burns/PycharmProjects/MasterThesis/data/IDFs/final_office_IdealLoad_summer_2023.idf"
simulation_dir = "simulations_2050" 

# Sicherstellen, dass das Ausgabeverzeichnis existiert
if not os.path.exists(simulation_dir):
    os.makedirs(simulation_dir)


# Define the problem for LHS
problem = {
    'num_vars': 3,
    'names': ['cooling_setpoint', 'people_in_room', 'cooling_load'],
    'bounds': [[15, 30],   # Cooling setpoint range
               [0, 6],    # People in room range
               [0, 1000]] # Cooling load range
}

# Generate LHS samples
samples = latin.sample(problem, 1000)

# Simulationen durchführen
for i, sample in enumerate(samples):
    cooling_setpoint = sample[0]
    people_in_room = sample[1]
    cooling_load = sample[2]
    print(i)
    output_idf_path = os.path.join(simulation_dir, "simulation_" + str(i)+ ".idf")
    output_simulation_dir = os.path.join(simulation_dir, f'result_{i}' + "/")
    # print(output_simulation_dir)

    if not os.path.exists(output_simulation_dir):
        os.makedirs(output_simulation_dir)

    # IDF-Datei anpassen
    modify_idf(template_idf_path, output_idf_path, cooling_setpoint, people_in_room, cooling_load)
    
    # Simulation ausführen
    # run_simulation(output_idf_path, output_simulation_dir)
    idf_name = "simulation_" + str(i) + ".idf"
    # print(idf_name)
    # run_simulation(idf_name, output_simulation_dir)
    run_simulation(output_idf_path, output_simulation_dir)


In [ ]:
# plot the results of the simulation runs 
string_mean_air_temp_name = 'THERMAL ZONE 1:Zone Mean Air Temperature [C](TimeStep)'
string_cooling_load_name = 'THERMAL ZONE 1 IDEAL LOADS AIR SYSTEM:Zone Ideal Loads Zone Total Cooling Rate [W](TimeStep)'


import os
import pandas as pd
import matplotlib.pyplot as plt

# Pfad zum Verzeichnis der Simulationsergebnisse
simulation_dir = 'simulations'

# Funktion zum Extrahieren der Ergebnisse aus den Simulationen
def extract_results(simulation_dir, num_simulations):
    results = []
    for i in range(num_simulations):
        result_dir = os.path.join(simulation_dir, f'result_{i}')
        print(result_dir)
        output_csv = os.path.join(result_dir, 'eplusout.csv')  # Assuming the results are in eplusout.csv

        if os.path.exists(output_csv):
            data = pd.read_csv(output_csv)
            indoor_temp = data['THERMAL ZONE 1:Zone Mean Air Temperature [C](TimeStep)'].mean()  # Adjust column name as needed
            cooling_load = data['THERMAL ZONE 1 IDEAL LOADS AIR SYSTEM:Zone Ideal Loads Zone Total Cooling Rate [W](TimeStep)'].sum()  # Adjust column name as needed
            results.append((indoor_temp, cooling_load))
        else:
            print(f"Result file {output_csv} not found.")
    return results

# Extrahieren der Ergebnisse
num_simulations = 9  # Anzahl der Simulationen
results = extract_results(simulation_dir, num_simulations)

# Konvertieren der Ergebnisse in ein DataFrame
df = pd.DataFrame(results, columns=['Indoor Temperature (C)', 'Cooling Load (W)'])

# Plotten der Ergebnisse
plt.figure(figsize=(10, 5))

# Histogramm der Innentemperaturen
plt.subplot(1, 2, 1)
plt.hist(df['Indoor Temperature (C)'], bins=20, color='blue', edgecolor='black')
plt.title('Distribution of Indoor Temperatures')
plt.xlabel('Indoor Temperature (C)')
plt.ylabel('Frequency')

# Histogramm der Kühllasten
plt.subplot(1, 2, 2)
plt.hist(df['Cooling Load (W)'], bins=20, color='green', edgecolor='black')
plt.title('Distribution of Cooling Loads')
plt.xlabel('Cooling Load (W)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


In [ ]:
# Pfad zum Verzeichnis der Simulationsergebnisse
output_file = 'eplusout.csv'  # Anpassung des Dateinamens

# Funktion zum Extrahieren der Zeitreihendaten
def extract_time_series(simulation_dir, num_simulations, num_days=7):
    all_results = []
    for i in range(num_simulations):
        result_dir = os.path.join(simulation_dir, f'result_{i}')
        output_csv = os.path.join(result_dir, output_file)  # Pfad zur CSV-Datei

        if os.path.exists(output_csv):
            data = pd.read_csv(output_csv)
            # Filterung der Daten für die ersten 7 Tage (Annahme: stündliche Daten)
            data = data.iloc[:num_days*24, :]
            indoor_temp = data[string_cooling_load_name]  # Anpassung des Spaltennamens
            time = pd.date_range(start='7/1/2022', periods=len(indoor_temp), freq='h')
            all_results.append(pd.DataFrame({'Time': time, 'cooling load (W)': indoor_temp}))
        else:
            print(f"Result file {output_csv} not found.")
    return all_results

# Extrahieren der Zeitreihendaten
num_simulations = 10  # Anzahl der Simulationen (für dieses Beispiel 10)
results = extract_time_series(simulation_dir, num_simulations)

# Plotten der Zeitreihen
plt.figure(figsize=(14, 7))
for i, result in enumerate(results):
    plt.plot(result['Time'], result['cooling load (W)'], label=f'Simulation {i+1}')

plt.title('cooling load (W) Over 7 Days for 10 Simulations')
plt.xlabel('Time')
plt.ylabel('cooling load (W)')
plt.legend()
plt.show()